In [42]:
# Basic DS stuff
import numpy as np
import pandas as pd
import json

# Web scraping
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import asyncio

# For labeling records, tracking files, and formatting
from datetime import datetime

# For Rick
import pickle

In [38]:
# Load data

with open('../data/raw/all_users', 'rb+') as file :
    all_users = pickle.load(file)

with open('../data/raw/game_mapped_users', 'rb+') as file :
    game_mapped_users = pickle.load(file)
    
with open('../data/raw/skipped_game_map_users', 'rb+') as file :
    skipped_users = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/skipped_game_map_users'

In [39]:
# Create vars

holding_set = set()

unmapped_users = list(all_users-game_mapped_users)

key = '83A112A706DF72CB0920A20F28F04252'

interval = 10

base_url = 'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key={}&steamid={}&format=json'

-------------
### This is the API version

In [40]:
async def get_games(user, url) :
    return (user, urlopen(url))

In [62]:


for i in range(0, len(unmapped_users), interval) :
    iter_max = min(i+interval, len(unmapped_users))
    users = [unmapped_users[i] for i in range(i, iter_max)]
    tasks = [get_games(user, base_url.format(key, user)) for user in users]
    responses = await asyncio.gather(*tasks)
    for response in responses :
        user = response[0]
        jason = json.load(response[1])
        games_list = jason['response']
        if len(games_list) > 0 :
            if games_list['total_count'] > 0 :
                for game in games_list['games'] :
                    appid = game['appid']
                    playtime_2weeks = game['playtime_2weeks']
                    playtime_forever = game['playtime_forever']
                    holding_set.add((user, appid, playtime_2weeks, playtime_forever))
        mapped_users.add(user)

    if i % 200 == 0 :
        with open('../data/raw/recently_played.pkl', 'rb+') as file :
            extant_set = pickle.load(file)
            total_set = extant_set.union(holding_set)
            pickle.dump(total_set, file)
            file.seek(0)
        print(f"Saved so far: {len(total_set)}")
        print(datetime.now())
        holding_set = set()


Saved so far: 3
2024-03-21 14:55:16.191151


CancelledError: 

In [25]:
# This works but is deprecated. Saved as a potential reference.

# counter = 0
# game_counts = set()

# for i in range(100) :

#     user = all_users[i]

#     url = f'http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key={key}&steamid={user}&format=json'
#     with urlopen(url) as response :
#         jason = json.load(response)
#     games_list = jason['response']
#     if len(games_list) > 0 :
#         if games_list['total_count'] > 0 :
#             print(jason)
#             print('\n')
#             counter +=1
#             game_counts.add(len(games_list))

# print(f'{counter} users had recently played games. Of them:')
# print(f'Avg games played: {sum(game_counts)/len(game_counts)}')
# print(f'Most games played: {max(game_counts)}')

{'response': {'total_count': 2, 'games': [{'appid': 646570, 'name': 'Slay the Spire', 'playtime_2weeks': 2350, 'playtime_forever': 268378, 'img_icon_url': '33ea124ea8c03a9ce7012d34c3b348a351612fca'}, {'appid': 2379780, 'name': 'Balatro', 'playtime_2weeks': 2015, 'playtime_forever': 3005, 'img_icon_url': 'b6018068070ab0e23561694c11f7950dd6f4c752'}]}}


{'response': {'total_count': 8, 'games': [{'appid': 2357570, 'name': 'Overwatch® 2', 'playtime_2weeks': 3240, 'playtime_forever': 31389, 'img_icon_url': 'da42bd294c941d5947b1bc2c2b2efa1313d36a91'}, {'appid': 1971650, 'name': 'OCTOPATH TRAVELER II', 'playtime_2weeks': 251, 'playtime_forever': 1857, 'img_icon_url': '7a90088ff3b521a0edcbd414fcd88361dd3d8b45'}, {'appid': 1364780, 'name': 'Street Fighter™ 6', 'playtime_2weeks': 12, 'playtime_forever': 4671, 'img_icon_url': '5e1421a150aaded0c7c175bc6f6843eab9a4d27d'}, {'appid': 3480, 'name': 'Peggle Deluxe', 'playtime_2weeks': 9, 'playtime_forever': 9, 'img_icon_url': '427a98a549c3813e13b406230